# 서울시 인구 분석

<img src='https://raw.githubusercontent.com/Jangrae/img/master/people2.png' width="650" align="left">

## 1. 라이브러리 불러오기

- 사용할 라이브러리를 불러옵니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

* 데이터 불러오기

    - 다음 경로의 파일을 읽어와 pop01, pop02, pop03 데이터프레임을 만듭니다.
    - 파일 경로1: https://raw.githubusercontent.com/Jangrae/csv/master/seoul_pop_h01.csv
    - 파일 경로2: https://raw.githubusercontent.com/Jangrae/csv/master/seoul_pop_h02.csv
    - 파일 경로3: https://raw.githubusercontent.com/Jangrae/csv/master/seoul_pop_h03.csv

In [2]:
pop1 = pd.read_csv('https://raw.githubusercontent.com/Jangrae/csv/master/seoul_pop_h01.csv')
pop1.head()

,year,k_male,k_female
0,1981,4160,4191
1,1982,4160,4191
2,1983,4160,4191
3,1984,4160,4191
4,1985,4160,4191


In [3]:
pop2 = pd.read_csv('https://raw.githubusercontent.com/Jangrae/csv/master/seoul_pop_h02.csv')
pop2.head()

,year,f_male,f_female
0,1985,7,6
1,1986,7,5
2,1987,6,5
3,1988,5,5
4,1989,6,5


In [4]:
pop3 = pd.read_csv('https://raw.githubusercontent.com/Jangrae/csv/master/seoul_pop_h03.csv')
pop3.head()

,year,household,older_65
0,1981,1915,246
1,1982,2001,260
2,1983,2116,260
3,1984,2246,275
4,1985,2338,211


## 2. 데이터 탐색

다음과 같은 정보 확인을 통해 처리할 대상 데이터를 이해합니다.
- 상/하위 데이터 확인
- 데이터프레임 크기 확인
- 열 이름, 데이터 형식, 값 개수 등 확인
- 기초 통계정보 확인
- 결측치 확인
- 범주형 데이터 확인
- 개별 열 값 상세 확인 등

**1) 데이터프레임 크기 확인**

- 세 개의 데이터프레임 크기(행 수, 열 수)를 확인합니다.

In [5]:
pop1.shape, pop2.shape, pop3.shape
# pop2가 행 4개가 적다 

((40, 3), (36, 3), (40, 3))

**2) year 최솟값, 최댓값 확인**

- 세 개의 데이터프레임 year열 최솟값, 최댓값 크기를 비교해 차이가 있는 지 각각 확인합니다.

In [6]:
pop1.loc[:,'year'].agg({'min', 'max'})

min    1981
max    2020
Name: year, dtype: int64

In [7]:
pop2.loc[:,'year'].agg({'min', 'max'})
# 없는 행 4개가 1981, 1982, 1983, 1984 인 거 같다

min    1985
max    2020
Name: year, dtype: int64

In [8]:
pop3.loc[:,'year'].agg({'min', 'max'})

min    1981
max    2020
Name: year, dtype: int64

**3) 결측치 확인**

- 세 개의 데이터프레임에 결측치가 있는 지 각각 확인합니다.

In [9]:
pop1.isna().sum()

year        0
k_male      0
k_female    0
dtype: int64

In [10]:
pop2.isna().sum()

year        0
f_male      0
f_female    0
dtype: int64

In [11]:
pop3.isna().sum()

year         0
household    0
older_65     0
dtype: int64

**4) 열 정보 확인**

- 세 개의 데이터프레임의 열 정보를 확인합니다.

In [12]:
pop1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   year      40 non-null     int64
 1   k_male    40 non-null     int64
 2   k_female  40 non-null     int64
dtypes: int64(3)
memory usage: 1.1 KB


In [13]:
pop2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   year      36 non-null     int64
 1   f_male    36 non-null     int64
 2   f_female  36 non-null     int64
dtypes: int64(3)
memory usage: 992.0 bytes


In [14]:
pop3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   year       40 non-null     int64
 1   household  40 non-null     int64
 2   older_65   40 non-null     int64
dtypes: int64(3)
memory usage: 1.1 KB


## 3. 데이터 전처리

전처리 과정에서 다음과 같은 처리를 할 수 있습니다.

- 결측치 처리
- 값 변경
- 열 추가
- 불필요한 열 제거
- 열 이름 변경
- 데이터 통합(연결 또는 조인)
- 가변수화 등

**1) 데이터 통합**

- concat을 사용해 연결하면 인덱스 값을 기준으로 연결되어 데이터가 어긋납니다.
- **year** 열을 기준으로 **outer** 방식으로 조인(병합)합니다.
- 외국인 정보가 1981 ~ 1984년이 누락되어 결측치가 발생할 것입니다.
- 세 개의 데이터프레임을 병합(조인)하여 pop 데이터프레임을 선언합니다.
- 이후의 모든 작업은 pop 데이터프레임을 대상으로 진행합니다.

In [15]:
temp = pd.merge(pop1, pop2, how='outer',on='year')
temp.head()

,year,k_male,k_female,f_male,f_female
0,1981,4160,4191,NaN,NaN
1,1982,4160,4191,NaN,NaN
2,1983,4160,4191,NaN,NaN
3,1984,4160,4191,NaN,NaN
4,1985,4160,4191,7.0,6.0


In [16]:
pop = pd.merge(temp, pop3, how='outer', on='year')
pop.head(10)

,year,k_male,k_female,f_male,f_female,household,older_65
0,1981,4160,4191,NaN,NaN,1915,246
1,1982,4160,4191,NaN,NaN,2001,260
2,1983,4160,4191,NaN,NaN,2116,260
3,1984,4160,4191,NaN,NaN,2246,275
4,1985,4160,4191,7.0,6.0,2338,211
5,1986,4899,4888,7.0,5.0,2428,305
6,1987,5000,4979,6.0,5.0,2518,329
7,1988,5156,5120,5.0,5.0,2658,349
8,1989,5305,5261,6.0,5.0,2817,363
9,1990,5321,5282,5.0,4.0,2820,363


**2) 결측치 확인**

- 결측치가 있는지 확인합니다.

In [17]:
pop.isna().sum()

year         0
k_male       0
k_female     0
f_male       4
f_female     4
household    0
older_65     0
dtype: int64

**3) 결측치 처리**

- 연도별 인구 현황이므로 임의의 값을 채우는 것이 바람직하지 않아 보입니다.
- 이후 값, 즉 1985년 값으로 채우는 것도 정확한 분석을 방해할 것 같습니다.
- 이에 결측치가 있는 1981년~1984년 행을 제거할 것입니다.

In [18]:
pop.dropna(axis=0, inplace=True)
pop.isna().sum()

year         0
k_male       0
k_female     0
f_male       0
f_female     0
household    0
older_65     0
dtype: int64

In [19]:
pop.shape

(36, 7)

**3) 열 추가**

- 이후 분석의 편의를 위해 다음과 같은 의미를 갖는 열을 추가하고자 합니다.
    - k_total = 전체 한국인 인구수
    - f_total = 전체 외국인 인구수
    - male = 전체 남자 인구수
    - female = 전체 여자 인구수
    - total = 전체 인구수
- 추가할 열에 대한 공식은 다음과 같습니다.
    - k_total = k_male + k_female
    - f_total = f_male + f_female
    - male = k_male + f_male
    - female = k_female + f_female
    - total = k_total + f_total
- 정리한 공식에 따라 데이터프레임에 열을 추가합니다.

In [20]:
pop['k_total'] = pop['k_male']+pop['k_female']
pop['f_total'] = pop['f_male']+pop['f_female']
pop['male'] = pop['k_male']+pop['f_male']
pop['female'] = pop['k_female']+pop['f_female']
pop['total'] = pop['k_total']+pop['f_total']
pop.head()

,year,k_male,k_female,f_male,f_female,household,older_65,k_total,f_total,male,female,total
4,1985,4160,4191,7.0,6.0,2338,211,8351,13.0,4167.0,4197.0,8364.0
5,1986,4899,4888,7.0,5.0,2428,305,9787,12.0,4906.0,4893.0,9799.0
6,1987,5000,4979,6.0,5.0,2518,329,9979,11.0,5006.0,4984.0,9990.0
7,1988,5156,5120,5.0,5.0,2658,349,10276,10.0,5161.0,5125.0,10286.0
8,1989,5305,5261,6.0,5.0,2817,363,10566,11.0,5311.0,5266.0,10577.0


**4) 열 순서 변경**

- 데이터 이해를 돕기 위해 다음과 같은 순서로 데이터프레임 열 순서를 변경합니다.
- year, household, total, male, female, k_total, k_male, k_female, f_total, f_male, f_female, older_65


In [21]:
pop = pop[['year', 'household', 'total', 'male', 'female', 'k_total', 'k_male', 'k_female', 'f_total', 'f_male', 'f_female', 'older_65']].head()

**5) 인덱스 초기화**

- 인덱스가 0부터 시작하는 일련 변호를 갖지 않는다면 인덱스를 초기화합니다.

In [22]:
pop.head()

,year,household,total,male,female,k_total,k_male,k_female,f_total,f_male,f_female,older_65
4,1985,2338,8364.0,4167.0,4197.0,8351,4160,4191,13.0,7.0,6.0,211
5,1986,2428,9799.0,4906.0,4893.0,9787,4899,4888,12.0,7.0,5.0,305
6,1987,2518,9990.0,5006.0,4984.0,9979,5000,4979,11.0,6.0,5.0,329
7,1988,2658,10286.0,5161.0,5125.0,10276,5156,5120,10.0,5.0,5.0,349
8,1989,2817,10577.0,5311.0,5266.0,10566,5305,5261,11.0,6.0,5.0,363


In [23]:
pop.reset_index(drop=True, inplace=True)
pop.head()

,year,household,total,male,female,k_total,k_male,k_female,f_total,f_male,f_female,older_65
0,1985,2338,8364.0,4167.0,4197.0,8351,4160,4191,13.0,7.0,6.0,211
1,1986,2428,9799.0,4906.0,4893.0,9787,4899,4888,12.0,7.0,5.0,305
2,1987,2518,9990.0,5006.0,4984.0,9979,5000,4979,11.0,6.0,5.0,329
3,1988,2658,10286.0,5161.0,5125.0,10276,5156,5120,10.0,5.0,5.0,349
4,1989,2817,10577.0,5311.0,5266.0,10566,5305,5261,11.0,6.0,5.0,363


## 4. 추가 전처리

* x, y 구분

    - x : feature
    - y : target <- total

In [29]:
target = 'total'
pop.shape

(5, 12)

In [25]:
X = pop.drop(target, axis=1)
y = pop.loc[:, target]

In [28]:
X.shape, y.shape

((5, 11), (5,))

* train test split

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

In [32]:
train_x.shape, train_y.shape

((3, 11), (3,))

* Scaling

    - min-max scaling

In [33]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
scaler = MinMaxScaler()

In [35]:
train_x = scaler.fit_transform(train_x)
test_x = scaler.fit_transform(test_x)

In [36]:
pd.DataFrame(train_x).describe()

,0,1,2,3,4,5,6,7,8,9,10
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.00,3.000000,3.000000
mean,0.583333,0.556019,0.622960,0.622700,0.623025,0.623290,0.622741,0.444444,0.50,0.333333,0.635965
std,0.520416,0.509328,0.543468,0.543292,0.543512,0.543692,0.543320,0.509175,0.50,0.577350,0.552684
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
25%,0.375000,0.334029,0.434441,0.434051,0.434537,0.434934,0.434112,0.166667,0.25,0.000000,0.453947
50%,0.750000,0.668058,0.868881,0.868101,0.869074,0.869869,0.868224,0.333333,0.50,0.000000,0.907895
75%,0.875000,0.834029,0.934441,0.934051,0.934537,0.934934,0.934112,0.666667,0.75,0.500000,0.953947
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00,1.000000,1.000000


In [37]:
pd.DataFrame(test_x).describe()

,0,1,2,3,4,5,6,7,8,9,10
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.0,2.000000
mean,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.0,0.500000
std,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.0,0.707107
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.0,0.250000
50%,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.0,0.500000
75%,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.0,0.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000


## 5. 모델링

* 모델 선언

In [38]:
from sklearn.linear_model import LinearRegression # 선형회귀
from sklearn.metrics import mean_squared_error # 잔차제곱의 평균

In [39]:
# 모델 선언
model = LinearRegression()

* 모델 학습

In [40]:
model.fit(train_x, train_y)

LinearRegression()

* 학습한 모델 기반으로 예측값 생성

In [42]:
test_pred = model.predict(test_x)
test_pred.shape

(2,)

In [44]:
test_pred

array([10615.35655954,  8577.02668107])

In [46]:
test_y

[2    9990.0
 1    9799.0
 Name: total, dtype: float64]

## 6. 평가

In [47]:
mean_squared_error(test_y, test_pred, squared=False)

970.641442223591